# Kinematic Data Augmention

This notebook demonstrates a data augmentation process for kinematic data, including the estimation of additional joint positions, angles, and distances, using geometric methods for both relative and absolute values, with the assumption that the performer is in an upright position.

### Required Modules

In [13]:
import os
import cv2
import math
import pandas as pd
import numpy as np

## Indexing by Seconds

For consistency and efficiency, we index the frames by seconds using the frames-per-second (fps) value. This allows us to divide the data into equal time intervals and facilitates the computation of features such as distances and velocities.

In [14]:
kd = pd.read_csv('./data/interim/contemporary_001/landmarks_rel.csv')
fps = int(kd['fps'][0])
kd = kd[::fps].reset_index(drop=True)
kd.tail(5)

,frame,fps,nose_x,nose_y,nose_z,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_x,left_eye_y,...,left_heel_z,right_heel_x,right_heel_y,right_heel_z,left_foot_index_x,left_foot_index_y,left_foot_index_z,right_foot_index_x,right_foot_index_y,right_foot_index_z
27,810,30.0,0.498949,0.329888,-0.184884,0.497939,0.324411,-0.172670,0.499529,0.325006,...,0.012451,0.359450,0.920613,0.058124,0.650630,0.989615,-0.074555,0.365629,0.979559,-0.051518
28,840,30.0,0.475244,0.511374,-0.218917,0.480640,0.495980,-0.210376,0.483769,0.494165,...,0.064710,0.373186,0.967389,0.009535,0.628840,1.003726,-0.040801,0.338033,0.994027,-0.075324
29,870,30.0,0.567704,0.602691,-0.233505,0.578145,0.592307,-0.222292,0.578995,0.591339,...,0.074900,0.349946,0.959117,0.245602,0.612308,0.996352,0.015325,0.324406,0.991867,0.174376
30,900,30.0,0.553052,0.540006,-0.419900,0.559146,0.528141,-0.395575,0.559706,0.528495,...,0.149811,0.234060,0.944354,-0.077509,0.546164,0.980541,0.106084,0.239721,1.026660,-0.191311
31,930,30.0,0.496254,0.260249,-0.259014,0.500789,0.245645,-0.232043,0.503714,0.247162,...,0.232146,0.271552,0.971011,0.041474,0.463709,1.044321,0.134770,0.257684,1.008112,-0.101646


## Augmenting Joint Positions

MediaPipe's current implementation involves using 32 landmarks to identify joints (as shown in a reference picture). In order to gain a more comprehensive understanding of body alignment and posture during performances, we have estimated additional joint positions, which can be helpful in achieving this goal.

![Image](https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png)

### Forehead Joint

This computation estimates the position of the forehead joint using the average position of the outer corners of both eyes in both relative and absolute kinematic data.

In [15]:
kd.loc[:, 'forehead_x'] = (kd['left_eye_outer_x'] + kd['right_eye_outer_x']) / 2
kd.loc[:, 'forehead_y'] = (kd['left_eye_outer_y'] + kd['right_eye_outer_y']) / 2
kd.loc[:, 'forehead_z'] = (kd['left_eye_outer_z'] + kd['right_eye_outer_z']) / 2

### Torso Joint

This computation estimates the location of the torso joint by averaging the positions of the left and right shoulders and left and right hips.

In [16]:
kd.loc[:, 'torso_x'] = (kd['left_shoulder_x'] + kd['right_shoulder_x'] + kd['left_hip_x'] + kd['right_hip_x']) / 4
kd.loc[:, 'torso_y'] = (kd['left_shoulder_y'] + kd['right_shoulder_y'] + kd['left_hip_y'] + kd['right_hip_y']) / 4
kd.loc[:, 'torso_z'] = (kd['left_shoulder_z'] + kd['right_shoulder_z'] + kd['left_hip_z'] + kd['right_hip_z']) / 4

## Augmenting Joint Data with Angles, Distance, and Velocity
We can extract valuable information about body alignment and posture during a performance by augmenting the joint data with joint angles, distances, and velocities. To compute joint angles, we use the dot product of the joint vectors and the arccosine function to convert it into degrees. For distances, we use the Euclidean distance between the joint positions. To compute velocities, we take the difference in distance between consecutive frames and multiply it by the frames-per-second (fps) value. These computed features are based on the connections between limbs in the human body.

And here are the formulas for angle, distance, and velocity in LaTeX:

Joint angle: $\theta = \cos^{-1}\left(\frac{\vec{v_1}\cdot\vec{v_2}}{\left|\vec{v_1}\right|\left|\vec{v_2}\right|}\right)$

Joint distance: $d = \left|\vec{v_1} - \vec{v_2}\right|$

Joint velocity: $v = (d_t - d_{t-1}) \cdot \text{fps}$

where $\vec{v_1}$ and $\vec{v_2}$ are the joint vectors at time $t$ and $d_t$ and $d_{t-1}$ are the distances between the joints at time $t$ and $t-1$, respectively.

Source: [Vector Operations](https://www.khanacademy.org/math/algebra-home/alg-vectors)

### Defining Connected Joints
The joint pairs defined represent connections between limbs in the human body.

In [17]:
joint_pairs = [('right_shoulder', 'right_elbow'), ('right_elbow', 'right_wrist'), ('right_hip', 'right_knee'), ('right_knee', 'right_ankle'), ('right_ankle', 'right_foot_index'), ('left_shoulder', 'left_elbow'), ('left_elbow', 'left_wrist'), ('left_hip', 'left_knee'), ('left_knee', 'left_ankle'), ('left_ankle', 'left_foot_index'), ('forehead', 'torso')]

### Computing Angles and Magnitude

In [18]:
for joint in joint_pairs:
    angles = []
    distances = []
    velocities = []
    for i in range(len(kd)):
        joint1 = np.array([kd[f"{joint[0]}_x"].iloc[i], kd[f"{joint[0]}_y"].iloc[i], kd[f"{joint[0]}_z"].iloc[i]])
        joint2 = np.array([kd[f"{joint[1]}_x"].iloc[i], kd[f"{joint[1]}_y"].iloc[i], kd[f"{joint[1]}_z"].iloc[i]])     
        dot_product = np.dot(joint1, joint2)
        magnitude1 = np.linalg.norm(joint1)
        magnitude2 = np.linalg.norm(joint2)
        distance = np.linalg.norm(joint1 - joint2)
        angle = np.degrees(np.arccos(dot_product / (magnitude1 * magnitude2)))

        angles.append(angle)
        distances.append(distance)

        if i == 0:
            velocity = 0
            velocities.append(velocity)
        else:
            velocity = (distances[i] - distances[i-1]) * fps
            velocities.append(velocity)

    kd[f"a_{joint[0]}_{joint[1]}"] = angles
    kd[f"d_{joint[0]}_{joint[1]}"] = distances
    kd[f"v_{joint[0]}_{joint[1]}"] = velocities

## Rate of Change

The rate of change of a quantity measures how quickly that quantity is changing over time. It can be approximated by computing the difference between consecutive values of the quantity and dividing by the time difference between those values.

\begin{equation}
\text{rate of change}[i] = \frac{x[i] - x[i-1]}{t[i] - t[i-1]}
\end{equation}

[Reviewer on Khan Academy](https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:average-rate-of-change/v/introduction-to-average-rate-of-change)

In [20]:
joints = ["forehead", "torso", "right_shoulder", "left_shoulder", "right_elbow", "left_elbow", "right_knee", "left_knee", "right_ankle", "left_ankle"]
joints_x = [joint + "_x" for joint in joints]
joints_y = [joint + "_y" for joint in joints]
joints_z = [joint + "_z" for joint in joints]

for joint_x, joint_y, joint_z in zip(joints_x, joints_y, joints_z):
    diff_x = kd[joint_x].diff().fillna(0)
    diff_y = kd[joint_y].diff().fillna(0)
    diff_z = kd[joint_z].diff().fillna(0)
    kd[f"diff_{joint_x}"] = diff_x
    kd[f"diff_{joint_y}"] = diff_y
    kd[f"diff_{joint_z}"] = diff_z
    
kd.head()

,frame,fps,nose_x,nose_y,nose_z,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_x,left_eye_y,...,diff_right_knee_z,diff_left_knee_x,diff_left_knee_y,diff_left_knee_z,diff_right_ankle_x,diff_right_ankle_y,diff_right_ankle_z,diff_left_ankle_x,diff_left_ankle_y,diff_left_ankle_z
0,0,30.0,0.501249,0.210489,-0.084651,0.508947,0.194841,-0.062590,0.514194,0.195405,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,30,30.0,0.495496,0.219194,-0.313203,0.502713,0.201868,-0.287782,0.507673,0.202800,...,-0.005352,-0.007760,0.007567,0.064296,-0.011314,-0.011690,0.036287,-0.008395,-0.003433,0.026202
2,60,30.0,0.485273,0.219256,-0.280624,0.492189,0.202155,-0.253160,0.496962,0.202840,...,-0.003858,-0.009441,0.010817,0.000331,-0.016307,0.037791,0.015470,-0.012651,0.042522,0.035923
3,90,30.0,0.486556,0.216945,-0.248385,0.493592,0.200263,-0.220545,0.498182,0.200760,...,0.003958,0.001870,-0.001290,0.001910,0.000252,0.004040,-0.010962,0.003609,-0.016371,-0.030781
4,120,30.0,0.433745,0.230009,-0.101605,0.439995,0.209614,-0.086546,0.444377,0.208915,...,0.005218,-0.000306,-0.003044,-0.069447,0.004292,0.010251,0.017208,0.009121,0.044594,-0.118247


## Save Processed DataFrames

In [21]:
kd.to_csv('./data/processed/contemporary_001.csv')

## Absolute Values

In [ ]:
h, w = 360, 640
kd_abs = kd.copy()
kd_abs.loc[:, kd_abs.columns.str.endswith('_x')] *= w
kd_abs.loc[:, kd_abs.columns.str.endswith('_y')] *= h
kd_abs.loc[:, kd_abs.columns.str.endswith('_z')] = h - kd_abs.loc[:, kd_abs.columns.str.endswith('_z')]
kd_abs.to_csv('./data/processed/contemporary_001_abs.csv', index=False)

## Batch Processing

For more details about the data augmentation processing, please refer to the code in [/src/data/augmentation.py](https://github.com/kayesokua/gestures/blob/main/src/data/augmentation.py)

In [1]:
from src.data.augmentation import batch_data_augmentation
batch_data_augmentation("./data/processed")

contemporary_007 has been processed!
contemporary_006 has been processed!
contemporary_004 has been processed!
contemporary_005 has been processed!
contemporary_001 has been processed!
classical_ballet_001 has been processed!
contemporary_002 has been processed!
contemporary_003 has been processed!
chinese_fan_001 has been processed!
contemporary_008 has been processed!
filipino_folk_001 has been processed!
Elapsed time: 0.717 seconds


## Summary
* The kinematic data has been indexed by second based on the fps value.
* Augmentation of the data includes new joint positions (torso, forehead), angles, distances, velocities of connected joints (connected limbs), and rate of change added for 10 joints (with the potential to add more).
* Further iterations and analysis may lead to changes in the data augmentation process.
   
## Resources
For a comprehensive list of resources, [please see here](https://github.com/kayesokua/gestures/blob/main/references/README.md).